Ok so lets gooo fire emoji

In [33]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

label_map = {
    "Normal": 0,
    "Depression": 1,
    "Suicidal": 2,
    "Anxiety": 3,
    "Stress": 4,
    "Bi-Polar": 5,
    "Personality Disorder": 6
}


db = pd.read_csv('Sentiment_analysis_dataset(in).csv')


In [34]:
db = db.dropna(subset=['Statement', 'Status'])
statement = db['Statement'].tolist()
status = db['Status'].tolist()

print(db.head(20))

                                            Statement      Status
0   life doesn’t feel worth it that’s kind of it? ...  Depression
1   This life sucks and if it were for my religiou...  Depression
2   its been 9 months now for our marriage and she...      Stress
3   I do not feel particularly sad or anxious or a...  Depression
4   I am taking Venlafaxine. it is an SSRI. does n...  Depression
5   I (17M) have come to the realisation that My d...  Depression
6   been on fluoxetine for year but anyway there s...  Depression
7   even a four day week seems too long i want to ...      Normal
8   I often had suicidal thoughts, recently, with ...    Suicidal
9   This is the first time facing Pisces. Extro pa...      Normal
10  being an orphan at your early stages of life b...  Depression
11  The dread of thinking you have cancer... again...     Anxiety
12  I (30F) married my husband (31M) about 6 month...      Stress
13           What day is Eid Al-Fitr if you may know?      Normal
14  Free c

In [35]:
train_texts, val_texts, train_labels, val_labels = train_test_split(db['Statement'], db['Status'], test_size=0.2, random_state=42)

print(f"training data size: {len(train_texts)}")
print(f"Testing data size: {len(val_texts)}")
print(db.iloc[2542])

training data size: 29500
Testing data size: 7375
Statement    ogbenidipo iwa were gba we see the reason you ...
Status                                              Depression
Name: 2560, dtype: object


In [36]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True,padding=True)

print(type(train_texts))
print(type(train_texts[0]))


<class 'pandas.core.series.Series'>
<class 'str'>


In [37]:
print(train_texts.head())


36248    i want everyone to feel my pain i am sick of e...
26254    Home. Had uni today, got results back :HD, YEH...
8331                                          really? why?
26739    I am not sure how much longer I can last. I al...
18584    I feel so trapped in my mind, the feeling that...
Name: Statement, dtype: object


In [ ]:
import torch

class MentalHealthDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        self.label_map = label_map

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        label = self.labels[idx]
        #item["labels"] = torch.tensor(self.label_map[idx])
        item["labels"] = torch.tensor(self.label_map[label])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MentalHealthDataset(train_encodings, train_labels)
val_dataset = MentalHealthDataset(val_encodings, val_labels)

Length of labels: 29500
Length of encodings: 2
Length of labels: 7375
Length of encodings: 2


In [56]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 7)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

KeyError: "Label 'Depression' not found in label_map."